In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

7.1 处理缺失数据

检查是否为缺失数据

In [2]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [3]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

7.1.1 滤除缺失数据

dropna 返回一个仅含非空数据和索引值的 Series

In [5]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

等价于

In [6]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

DataFrame 使用 dropna 默认丢弃任何含有缺失值的行

In [7]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [8]:
cleaned

,0,1,2
0,1.0,6.5,3.0


传入 how='all' 将只丢弃全为 NA 的行

In [9]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


丢弃列,只需设置 axis=1

In [10]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [11]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


保留至少有 n 个非 NA 数的行

In [12]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [13]:
df.dropna()

,0,1,2
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [19]:
df.dropna(thresh=2)

,0,1,2
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


7.1.3 填充缺失数据

使用 fillna 将缺失值替换为常数值

In [20]:
df.fillna(0)

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


使用字典填充

In [21]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.204708,0.500000,0.000000
1,-0.555730,0.500000,0.000000
2,0.092908,0.500000,0.769023
3,1.246435,0.500000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


fill默认会返回新对象,但也可以对现有对象进行就地修改

In [22]:
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


对 reindexing 有效的插值方法也可用与 fillna

In [25]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,NaN,1.343810
3,-0.713544,NaN,-2.370232
4,-1.860761,NaN,NaN
5,-1.265934,NaN,NaN


In [26]:
df.fillna(method='ffill')

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.124121,1.343810
3,-0.713544,0.124121,-2.370232
4,-1.860761,0.124121,-2.370232
5,-1.265934,0.124121,-2.370232


In [27]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.124121,1.343810
3,-0.713544,0.124121,-2.370232
4,-1.860761,NaN,-2.370232
5,-1.265934,NaN,-2.370232


使用平均值或者中位数填充

In [28]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

7.2 数据转换

7.2.1 转移重复数据

In [29]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


检查是否存在重复行

In [30]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

drop_duplicates 返回一个 DataFrame,重复的数组会删除

In [31]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


只过滤某一列重复的值

In [34]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


重复值默认保留第一个,传入 keep='last' 重复值保留最后一个

In [35]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


7.2.2 利用函数或映射进行数据转换

In [37]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


不同肉类到动物的映射

In [38]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

将 food 中的字母全部转换为小写

In [39]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [40]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


可以用一个函数完成以上全部工作

In [41]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

7.2.3 替换值

replace 提供了一种实现替换值更简单、更灵活的方式

In [43]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [44]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

一次替换多个值

In [45]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

多个值替换多个值

In [46]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

传入值也可以是字典

In [47]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

7.2.4 重命名索引

In [48]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [49]:
transform = lambda x: x[:4].upper()
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


创建数据集的转换版

In [50]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


rename 结合字典实现对部分轴标签更新

In [51]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


rename 实现复制 DataFrame 并对其索引和列标签进行赋值

In [52]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


7.2.5 离散化和面元划分

In [53]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

cut 的底层含有一个表示不同分类名词的类型数组,以及一个 codes 属性中的年龄数据的标签

In [54]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [55]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [56]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

那边是闭端可以通过 right=False 进行修改

In [57]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

通过传递一个裂变或数组到 labels,设置自己的面元名词

In [58]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

将一些均匀分布的数据分成四组

In [59]:
data = np .random.randn(20)
pd.cut(data, 4, precision=2)#precision=2限定小数只有两位

[(-0.5, 0.42], (-2.36, -1.43], (-0.5, 0.42], (-2.36, -1.43], (-1.43, -0.5], ..., (0.42, 1.35], (-1.43, -0.5], (-2.36, -1.43], (-0.5, 0.42], (0.42, 1.35]]
Length: 20
Categories (4, interval[float64]): [(-2.36, -1.43] < (-1.43, -0.5] < (-0.5, 0.42] < (0.42, 1.35]]

qcut 基于使用数据的样本分位数,因此可以得到大小基本相等的面元

In [60]:
data = np.random.randn(1000)
cats = pd.qcut(data, 4)
cats

[(-0.684, -0.029], (0.62, 3.928], (-0.029, 0.62], (-0.029, 0.62], (0.62, 3.928], ..., (-2.9499999999999997, -0.684], (0.62, 3.928], (-0.684, -0.029], (-2.9499999999999997, -0.684], (0.62, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.029] < (-0.029, 0.62] < (0.62, 3.928]]

In [61]:
pd.value_counts(cats)

(0.62, 3.928]                    250
(-0.029, 0.62]                   250
(-0.684, -0.029]                 250
(-2.9499999999999997, -0.684]    250
dtype: int64

也可以传递自定义的分位数

In [62]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-1.187, -0.029], (-0.029, 1.286], (-0.029, 1.286], (-0.029, 1.286], (-0.029, 1.286], ..., (-2.9499999999999997, -1.187], (-0.029, 1.286], (-1.187, -0.029], (-1.187, -0.029], (-0.029, 1.286]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -1.187] < (-1.187, -0.029] < (-0.029, 1.286] < (1.286, 3.928]]

7.2.6 检测和过滤异常值

In [63]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.002544,-0.051827,0.049433,0.025575
std,0.995232,0.998311,0.996714,1.007200
min,-3.745356,-3.428254,-3.645860,-3.184377
25%,-0.687373,-0.747478,-0.598600,-0.612162
50%,-0.022158,-0.088274,0.047101,-0.013609
75%,0.699046,0.623331,0.756646,0.690847
max,2.735527,3.366626,2.653656,3.525865


某列中绝对值大小超过 3 的值

In [65]:
col = data[2]
col[np.abs(col)>3]

543   -3.548824
802   -3.645860
Name: 2, dtype: float64

选出含有绝对值超过 3 的行

In [67]:
data[(np.abs(data)>3).any(1)]

,0,1,2,3
41,-3.399312,-0.974657,-0.685312,-0.645858
59,-0.817649,0.050188,1.951312,3.260383
136,-3.745356,-1.520113,-0.346839,-0.696918
234,-1.341493,-0.293333,-0.242459,-3.056990
258,0.425384,-3.428254,-0.296336,-0.439938
321,-0.085420,0.275144,1.179227,-3.184377
543,-0.150923,-0.362528,-3.548824,1.553205
635,1.397822,3.366626,-2.372214,0.851010
781,0.582317,-0.658090,-0.207434,3.525865
802,-0.080332,0.599947,-3.645860,0.255475


将值限制在区间-3 到 3 以内

In [68]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.001399,-0.051765,0.050628,0.025030
std,0.991414,0.995761,0.992686,1.003955
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.687373,-0.747478,-0.598600,-0.612162
50%,-0.022158,-0.088274,0.047101,-0.013609
75%,0.699046,0.623331,0.756646,0.690847
max,2.735527,3.000000,2.653656,3.000000


In [69]:
np.sign(data).head()

,0,1,2,3
0,-1.0,1.0,1.0,-1.0
1,1.0,-1.0,1.0,1.0
2,1.0,-1.0,-1.0,-1.0
3,1.0,-1.0,-1.0,1.0
4,-1.0,-1.0,1.0,-1.0


7.2.7 排列和随机采样

随机重排序 permutation

In [70]:
df = pd.DataFrame(np.arange(20).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

array([4, 0, 3, 2, 1])

In [71]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [72]:
df.take(sampler)

,0,1,2,3
4,16,17,18,19
0,0,1,2,3
3,12,13,14,15
2,8,9,10,11
1,4,5,6,7


sample 随机选取若干行

In [73]:
df.sample(n=3)

,0,1,2,3
2,8,9,10,11
4,16,17,18,19
0,0,1,2,3


可以通过替换的方式产生样本

In [74]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

2   -1
0    5
3    6
1    7
4    4
0    5
4    4
0    5
4    4
4    4
dtype: int64

7.2.8 计算指标/哑变量

DataFrame 的某一列中含有 k 个不同的值,则可以派生出一个 k 列矩阵或 DataFrame(其值全为 1 或 0)

In [76]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [77]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


列明添加一个前缀

In [80]:
dummies = pd.get_dummies(df['key'], prefix='key')
dum

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [83]:
df_with_dumy = df[['data1']].join(dummies)
df_with_dumy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


含有多个分类的情况

In [84]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames)
movies[:10]

<ipython-input-84-48314dfb286d>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_table('datasets/movielens/movies.dat', sep='::',


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


要为每个 genre 添加指标变量就需要做一些数据规整操作,首先抽取出不同的 genre 值

In [86]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

构造指标 DataFrame

In [88]:
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


计算每个类型的列索引

In [89]:
gen = movies.genres[0]
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2])

根据索引值,使用 iloc 设定值

In [90]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

在将其余 movies 合并起来

In [91]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

movie_id                                      1
title                          Toy Story (1995)
genres              Animation|Children's|Comedy
Genre_Animation                               1
Genre_Children's                              1
                               ...             
Genre_War                                     0
Genre_Musical                                 0
Genre_Mystery                                 0
Genre_Film-Noir                               0
Genre_Western                                 0
Name: 0, Length: 21, dtype: object

结合 get_dummies 和诸如 cut 之类的离散化函数

In [92]:
np.random.seed(12345)
values = np.random.rand(10)
values

array([0.9296, 0.3164, 0.1839, 0.2046, 0.5677, 0.5955, 0.9645, 0.6532,
       0.7489, 0.6536])

In [93]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


7.3 字符串操作

7.3.1 字符串对象方法

以逗号分隔的字符串可以用 split 拆分成数段

In [94]:
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

split 常常与 strip 一起使用,以去除空白符

In [95]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

利用加法,可以将这些子字符串以双冒号分隔符的形式连接起来

In [96]:
first, second, third = pieces
first + '::' + second + "::" + third

'a::b::guido'

向字符串'::'的 jion 方法传入一个列表或元组

In [99]:
'::'.join(pieces)

'a::b::guido'

子串定位,index 找不到返回异常,find 找不到返回-1

In [100]:
'guido' in val

True

In [102]:
val.index(',')

1

In [103]:
val.find(':')

-1

子串次数

In [104]:
val.count(',')

2

replace 用来替换,也用来删除

In [105]:
val.replace(',', '::')

'a::b::  guido'

In [106]:
val.replace(',', '')

'ab  guido'

7.3.2 正则表达式

描述一个或多个空白符的 regex 是\s+

In [107]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

可以用 re.compile 自己编译 regexg 以得到一个可重用的regex 对象

In [109]:
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

得到匹配 regex 的所有模式,可以使用 findall 方法

In [110]:
regex.findall(text)

['    ', '\t ', '  \t']

可以识别大部分电子邮件地址的正则表达式

In [111]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

对 text 使用 findall 将得到一组电子邮件地址

In [112]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

search 方法返回第一个电子邮件地址

In [114]:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [115]:
text[m.start():m.end()]

'dave@google.com'

regex.match 则将返回 None,它只匹配出现在字符串开头的模式

In [116]:
print(regex.match(text))

None


sub 方法可以将匹配到的模式替换为指定字符串,并返回所得到的的新字符串

In [117]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



将地址分成三个部分

In [121]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

使用 group 方法返回一个由模式各段组成的元组

In [123]:
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

findall 会返回一个元组列表

In [125]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

\1第一个匹配的组,\2 第二个匹配的组

In [126]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



7.3.3 pandas 的矢量化字符串函数

含有字符串的列还含有缺失数据

In [127]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [128]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

使用 str.contains 检查各个电子邮件地址是否还有某个字符串

In [129]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

也可以使用正则表达式

In [130]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [131]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

矢量化元素获取

In [132]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [136]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object